In [4]:
from github import Github
import pickle
import pandas as pd
import pprint
pd.set_option('display.max_rows', 150)
pp = pprint.PrettyPrinter(depth=6)

In [9]:
github_token = "f8f52658ba126656a20398edf8e1bce05fc089cc"

In [10]:
#using an access token
g = Github(github_token)
repo = g.get_repo("SonarSource/sonarqube")

#returned commits in a paginated list
commits = repo.get_commits()

commits_list = []
parents = []
i=0
for commit in commits:
    i+=1
    commits_list.append((commit.sha, commit.commit.message, commit.commit.author.date))
    for parent in commit.commit.parents:
        parents.append((commit.sha, parent.sha))

len(commits_list)

# Create pandas df
parentsDf = pd.DataFrame(parents, columns=["commit_sha", "parent_sha"])
parentsDf

(parentsDf.groupby("commit_sha").count().parent_sha == 1).sum()

# Create pandas df
commitsDf = pd.DataFrame(commits_list, columns=["Sha", "Message", "Date"])
commitsDf.Date = pd.to_datetime(commitsDf.Date)

commitsDf

pp.pprint(commits[0].commit.parents[0].sha)

pp.pprint(commits_list[0].__dict__)

# Dump tags objects retrieved from Github APIs

tags = repo.get_tags()
f = open('pkl/tags.pkl', 'wb')
for tag in tags:
    pickle.dump(tag, f)

tmp = []
for tag in tags:
    tmp.append((tag.name, tag.commit.sha, repo.get_commit(sha=tag.commit.sha).commit.author.date))

# Create a file with tags pandas df
tagsDf = pd.DataFrame(tmp, columns=["Tag", "Sha", "Date"])
#tagsDf.to_csv("csv/tags.csv")

In [12]:
tagsDf = pd.read_csv("../csv/tags.csv", index_col=0)
tagsDf.Date = pd.to_datetime(tagsDf.Date)

In [13]:
import datetime
tagsDf[tagsDf.Date > pd.to_datetime(datetime.date(2015,3,10))]

,Tag,Sha,Date
1,8.4.2.36762,2de1f343e3dbd9e88007b42e475512677c04eb8a,2020-08-27 14:11:31
2,8.4.1.35646,a0d372f704998d38856c26e28662fd7de062f873,2020-07-14 08:13:18
3,8.4.0.35506,74b4533439f380827d68e8c01667eeaecad89fc7,2020-07-03 07:09:27
4,8.3.1.34397,b77ce511a2039330aea8832ad12e86a8cfa9ff7d,2020-05-07 12:09:22
5,8.3.0.34182,9c58e36328ae79ad3c36c13aa253879351ce3613,2020-04-28 13:54:13
6,8.2.0.32929,749367809478e5b7f60c6ef952436db9789b5882,2020-02-25 14:32:24
7,8.1.0.31237,65a7a7c34de5bff9aed7c3e58eb6dcc87ae34b57,2019-12-17 12:29:34
8,8.0,87ca68d63f4afd37d74b2f454430dfde9e862c6a,2019-10-15 08:15:12
9,7.9.4,2f8d99a55899105cbf3e3170194c614c2a6ea4d7,2020-07-23 14:40:10
10,7.9.3,898a79cc7df01a9603c17f92b93e46ec62bc77d2,2020-03-24 07:49:27


Get commits differences between two tags
https://stackoverflow.com/questions/40293833/how-to-get-all-commits-in-a-git-tag-through-github-api

In [1]:
import requests

_BASE_URL = "https://api.github.com/"

headers = { \
    "Content-Type": "application/json", \
    "Accept": "application/json",
    "Travis-API-Version": "3",
    "Authorization": "token {}".format(githubToken)
}

def _request(method="get", endpoint="", headers=headers, **kwargs):
    """ Wrapper around requests.get and requests.post """

    methods = {
        "get": requests.get,
        "patch": requests.patch,
        "post": requests.post
    }

    if method in methods:
        request = methods[method]
    else:
        request = methods["get"]

    return request(_BASE_URL + endpoint, headers=headers, **kwargs)

In [17]:
def get_compare_tags(owner, repo, tag1, tag2):
    response = _request(f"{owner}/{repo}/{tag1}...{tag2}")
    return response

In [20]:
get_compare_tags("SonarSource", "sonarqube", "8.4.2.36762", "8.4.1.35646")

<Response [200]>